In [103]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: C:\Users\52477\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [104]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pandas as pd
import time
import requests
import os

# Load Enviroment Variables

In [105]:
load_dotenv()
chromedriver_path = os.getenv('CHROMEDRIVER')
email = os.getenv('EMAIL')
password = os.getenv('PASSWORD')

# Functions

In [106]:
def start_session(login_url):
    # Launch the browser with Selenium
    driver = webdriver.Chrome(executable_path=chromedriver_path)

    # Open the login page
    driver.get(login_url)

    # Wait until the login fields are available on the page
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'user[email]')))

    # Find the email and password fields by their ID
    email_field = driver.find_element(By.ID, 'user[email]')
    password_field = driver.find_element(By.ID, 'user[password]')

    # Enter the credentials (replace with your actual email and password)
    email_field.send_keys(email)  # Replace with your email
    password_field.send_keys(password)  # Replace with your password

    # Find and click the login button by its class name
    submit_button = driver.find_element(By.CLASS_NAME, 'button-primary')
    submit_button.click()
    
    # Return the driver object (which now represents the logged-in session)
    return driver

def scraping(files, driver):
    # Check if the folder exists, if not, create it
    if not os.path.exists('courses_web_page'):
       os.makedirs('courses_web_page')

    # Loop through each course in the 'files' DataFrame
    for idx in range(files.shape[0]):
        file = files.iloc[idx]
        url = file['url']

        # Check if the file type is either 'free_course' or 'paid_course'
        if file['type'] == 'free_course' or file['type'] == 'paid_course':  
            driver.get(url)
            # Wait for the page to load the element with class 'fr-view' (it could be the main content area)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'fr-view')))

            # Get the page source (HTML) from the loaded page
            html = driver.page_source

            # Create a BeautifulSoup object using the HTML source from Selenium
            soup = BeautifulSoup(html, 'html.parser')

            # Extract all the text content from the page using BeautifulSoup
            text = soup.get_text()

            # Split the extracted text into individual lines
            lines = text.splitlines()

            # Initialize variables to store the start and end positions of the desired text sections
            start_index = None
            end_index = None

            for i, line in enumerate(lines):
                if 'Descripción' in line:
                    start_index = i
                    break 

            for i, line in enumerate(lines):
                if '¡Vamos con todo!' in line:
                    end_index = i
                    break

            if start_index is not None and end_index is not None:
                section_text = '\n'.join(lines[start_index:end_index]).strip()
            else:
                section_text = "Sections aren't found."
            
            with open('courses_web_page/' +str(idx) + '.txt', 'w',  encoding='utf-8') as archivo:
                archivo.write(section_text)

        # Check if the file type is 'free_tutorial'        
        elif file['type'] == 'free_tutorial':
            driver.get(url)
            # Wait for the page to load the specific element for the tutorial section
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".section__subheading.section__subheading___f5420")))

            # Get the page source (HTML) from the loaded page
            html = driver.page_source

            # Create a BeautifulSoup object using the HTML source from Selenium
            soup = BeautifulSoup(html, 'html.parser')

            # Find all paragraph elements with the class 'section__subheading section__subheading___f5420'
            elements = soup.find_all('p', class_='section__subheading section__subheading___f5420')

            with open('courses_web_page/' + str(idx) + '.txt', 'w',  encoding='utf-8') as archivo:
                archivo.write(str(elements)[60:-5]) 

# Main

In [107]:
# Define the login URL for the website
login_url = 'https://cursos.desafiolatam.com/users/sign_in'

# Start a session by calling the 'start_session' function with the login URL
# This will open the browser and log in to the website
driver = start_session(login_url)
time.sleep(5)

# Read the CSV file containing a list of interesting courses into a DataFrame
files = pd.read_csv("InterestingLinks.csv")

# Call the 'scraping' function with the 'files' DataFrame and the 'driver' object
scraping(files, driver)

C:\Users\52477\AppData\Local\Temp\ipykernel_15840\774979625.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)
